做文本分析时候经常遇到同意多形词，如 ``Big Apple/New York/NN`` 都可能代指纽约市，当我们统计纽约市出现的次数的时候我们需要分别统计这三个词的数目并进行加总。

flashtext对于处理上面的问题非常擅长，而且运算速度特别快。清洗数据的速度，我们可以拿正则表达式来和flashtext作比较
![](img/速度.png)
我们发现运行正则表达式来清洗数据，速度会随着数据量线性下降，而flashtext的清洗性能基本保持不变。
## 一、安装

```
pip3 install flashtext
```
## 二、官方文档

```
https://flashtext.readthedocs.io/en/latest/
```

## 三、常用用法

### 3.1 不清洗，直接提取关键词

In [23]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('Big Apple')
keyword_processor.add_keyword('Bay Area')
keywords_found = keyword_processor.extract_keywords('I love big Apple and Bay Area.')
keywords_found

['Big Apple', 'Bay Area']

### 3.2 同义词指代，抽取关键词
像big apple 和new york都代指纽约，我们需要先清洗好数据，统一用一个词语代指纽约，再去抽取关键词语。这就用到 add_keyword方法。

In [11]:
from flashtext import KeywordProcessor
kw_processor = KeywordProcessor()
#给关键词处理器对象中加入待识别的关键词
kw_processor.add_keyword('Big Apple', 'New York')
kw_processor.add_keyword('Bay Area')
#对文本数据进行关键词提取
kws_found = kw_processor.extract_keywords('I love Big Apple and Bay Area.')
kws_found

['New York', 'Bay Area']

### 3.3 多个同义词指代
如果同义词太多，可以用字典构建映射关系。使用到的方法是add_keywords_from_dict

In [12]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {"java": ["java_2e", "java programing"],
                "product management": ["PM", "product manager"]}
#从字典中加入映射关系
keyword_processor.add_keywords_from_dict(keyword_dict)
#从列表中加入关键词
keyword_processor.add_keywords_from_list(["java", "python"])
keyword_processor.extract_keywords('I am a product manager for a java_2e and python platform')


['product management', 'java', 'python']

### 3.4 移除关键词
有的时候我们可能加错了关键词，想去除之前添加的关键词。这就用到remove_keyword/remove_keywords/remove_keywords_from_dict/remove_keywords_from_list

In [13]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {
    "java": ["java_2e", "java programing"],
    "product management": ["PM", "product manager"]}
keyword_processor.add_keywords_from_dict(keyword_dict)
print(keyword_processor.extract_keywords('I am a product manager for a java_2e platform'))

keyword_processor.remove_keyword('java_2e')

keyword_processor.remove_keywords_from_dict({"product management": ["PM"]})
keyword_processor.remove_keywords_from_list(["java programing"])
keyword_processor.extract_keywords('I am a product manager for a java_2e platform')

['product management', 'java']


['product management']

### 3.5 关键词个数
查看自定义的关键词个数

In [15]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {
    "java": ["java_2e", "java programing"],
    "product management": ["PM", "product manager"]
}
keyword_processor.add_keywords_from_dict(keyword_dict)
print(len(keyword_processor))

4


### 3.6 判断某个词是否存在于关键词处理器中

In [17]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('j2ee', 'Java')
print('j2ee' in keyword_processor)

True


In [21]:
print(keyword_processor.get_keyword('j2ee'))
print(keyword_processor.get_keyword('Java'))

Java
None


In [19]:
keyword_processor['colour'] = 'color'
print(keyword_processor['colour'])

color


### 3.7 获取关键词映射关系

In [22]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('j2ee', 'Java')
keyword_processor.add_keyword('colour', 'color')
keyword_processor.get_all_keywords()

{'j2ee': 'Java', 'colour': 'color'}

### 3.8 替换关键词

In [5]:
from flashtext import KeywordProcessor
kw_processor2 = KeywordProcessor()
# 给关键词处理器对象中加入待识别的关键词
kw_processor2.add_keyword('New Delhi', 'NCR region')
kw_processor2.add_keyword('Big Apple','New York')  # 注意顺序
#对文本数据进行关键词替换
new_sentence = kw_processor2.replace_keywords('I love Big Apple and new delhi.')
new_sentence

'I love New York and NCR region.'

### 3.9 关键词的位置
flashtext还能计算待考察词语的开始与结束的索引值

In [6]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('Big Apple', 'New York')
keyword_processor.add_keyword('Bay Area')
keywords_found = keyword_processor.extract_keywords('I love big Apple and Bay Area.', span_info=True)
keywords_found

[('New York', 7, 16), ('Bay Area', 21, 29)]

### 3.10 额外的信息
根据关键词，还能抽取一些额外的信息，如时间、位置等。但这些功能在中文中可能不太适用，英文问题不大。

In [7]:
from flashtext import KeywordProcessor
kp = KeywordProcessor()
kp.add_keyword('Taj Mahal', ('Monument', 'Taj Mahal'))
kp.add_keyword('Delhi', ('Location', 'Delhi'))
kp.extract_keywords('Taj Mahal is in Delhi.')

[('Monument', 'Taj Mahal'), ('Location', 'Delhi')]